In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import os
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import math
import matplotlib.pyplot as plt
import datetime as dt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

from tensorflow import keras

### 讀取資料

### Loading Data From Database(MySQL)

In [3]:
username = 'yuting'
password = pwd
host_port = host_port 
database = 'twfruits'

engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(username, password, host_port, database))
con = engine.connect()

In [4]:
fruit = "banana"
market = "台北二"
df= pd.read_sql(f"{fruit}_{market}", engine)
df

,日期,banana_平均價,banana_交易量,scarletbanana_平均價,guava_平均價,banana_前日平均價,banana_5日平均價
0,2011-01-01,31.4,9645.0,36.5,45.0,31.4,32.2
1,2011-01-02,32.0,12230.0,50.0,47.5,31.4,32.2
2,2011-01-03,32.2,12103.0,46.2,47.5,32.0,32.2
3,2011-01-04,32.4,11976.0,42.5,47.5,32.2,32.2
4,2011-01-05,33.2,7520.0,34.8,45.8,32.4,32.2
...,...,...,...,...,...,...,...
3868,2021-08-04,37.9,5550.0,35.2,31.4,35.2,37.7
3869,2021-08-05,40.7,5530.0,38.0,29.3,37.9,38.3
3870,2021-08-06,38.3,5891.0,38.6,32.9,40.7,38.0
3871,2021-08-07,46.1,4681.0,42.3,29.9,38.3,39.6


### 嘗試變因&結果
1. 改變測試集數量 => 0.2 to 0.1 (結果: 相同)

## 訓練set & 測試set

#### test data

In [5]:
test_samples = int(df.shape[0]*0.2)

In [6]:
#test_data = df[df["日期"] > '2019-12-31']
test_data = df.iloc[-test_samples: , 1:]

# 測試資料日期
#test_datelist = test_data["日期"]
#test_data.drop(columns="日期", inplace=True)

# 取得test_set(array)
test_set = test_data.values

#### train data

In [7]:
train_data = df.iloc[:-test_samples, 1:]

# 訓練資料日期
# train_datelist = train_data["日期"]
# train_data.drop(columns="日期", inplace=True)

# 取得train_set(array)
train_set = train_data.values

### 資料標準化

In [8]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### test

In [ ]:
sc_test = StandardScaler()
test_set_sc = sc_test.fit_transform(test_set[:, :])

sc_test_target = StandardScaler()
sc_test_target.fit_transform(test_set[:, 0:1])

In [ ]:
test_set_sc.shape

#### train

In [ ]:
#sc = MinMaxScaler(feature_range=(0,1))
sc_train = StandardScaler()
train_set_sc = sc_train.fit_transform(train_set[:, :])

sc_train_target = StandardScaler()
sc_train_target.fit_transform(train_set[:, 0:1])

In [ ]:
train_set_sc.shape

### 創造X、Y資料

#### test data

In [ ]:
X_test = []
y_test = []

n_future = 7
n_past = 30

# revised (0804)
for i in range(n_past, len(test_set_sc)-n_future+1): # 扣掉最後預測天數已符合y_test的length 
    X_test.append(test_set_sc[i-n_past:i, 1:])       # 利用前30天資料來預測後7天價格，特徵值排除banana價格本身
    y_test.append(test_set_sc[i:i+n_future, 0])      # 後7天價格
    
X_test, y_test = np.array(X_test), np.array(y_test) 
#y_test = y_test.reshape(X_test.shape[0], n_future, 1)

print("X_test's shape: {}".format(X_test.shape))
print("y_test's shape: {}".format(y_test.shape))

#### train data

In [ ]:
X_train = []
y_train = []

n_future = 7
n_past = 30

for i in range(n_past, len(train_set_sc)-n_future+1):
    X_train.append(train_set_sc[i-n_past:i, 1:]) # :
    y_train.append(train_set_sc[i:i+n_future, 0]) 
    
X_train, y_train = np.array(X_train), np.array(y_train)
#y_train = y_train.reshape(X_train.shape[0], n_future, 1)

print("X_train's shape: {}".format(X_train.shape))
print("y_train's shape: {}".format(y_train.shape))

In [ ]:
y_train[-1]

#### 建立模型

In [ ]:
model = Sequential()

n_steps = X_train.shape[1]
n_features = X_train.shape[2]
#n_outputs = n_future

# 1st layer 
# input_shape => n_step, n_feature 


model.add(LSTM(units=64, activation="relu", input_shape=(n_steps, n_features), return_sequences=True)) 

#model.add(RepeatVector(n_outputs))  # new

model.add(LSTM(units=32, activation="relu", return_sequences=False))          

model.add(Dropout(0.2))

#model.add(TimeDistributed(Dense(10, activation="relu")))

#model.add(TimeDistributed(Dense(7)))

model.add(Dense(y_train.shape[1], activation="linear"))          

#model.compile(optimizer="adam", loss="mean_squared_error")

model.summary() 

In [ ]:
model.compile(optimizer="adam", 
              loss="mean_squared_error")

In [ ]:
%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)
mcp = ModelCheckpoint(filepath='weights.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

tb = TensorBoard('logs')

In [ ]:
# fit model
history = model.fit(X_train, y_train, epochs=100, 
                    batch_size=32,
                    callbacks=[es, rlr, mcp, tb], 
                    validation_split=0.1, 
                    verbose=1)

In [ ]:
# plotting loss and validation loss
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

### predict train data

In [ ]:
# 預測未來&訓練集
#prediction_future = model.predict(trainX[-n_future:])

X = X_train
Y = y_train
prediction_train = model.predict(X)

In [ ]:
prediction_train = sc_train_target.inverse_transform(prediction_train)
Y = sc_train_target.inverse_transform(Y)

In [ ]:
prediction_train[0]

In [ ]:
A = prediction_train.reshape(len(X), -1).tolist()

In [ ]:
B = Y.reshape(len(Y), n_future).tolist()

In [ ]:
Y[0]#.shape

In [ ]:
prediction_train[0]#.shape

In [ ]:
A[-1]

In [ ]:
B[-1]

In [ ]:
rmse_output = []
for i in range(len(A)):
    rmse = mean_squared_error(A[i], B[i], squared=False)
    rmse_output.append(rmse)

In [ ]:
df_plot = pd.DataFrame({"rmse":rmse_output})
df_plot.plot()

In [ ]:
t = -150

In [ ]:
df_price_train = pd.DataFrame({"predicted price": A[t],
                              "Actual price": B[t]})

In [ ]:
df_price_train.plot()

### predict test data 

In [ ]:
#n_future = 1
X = X_test
y = y_test

In [ ]:
prediction_test = model.predict(X)

In [ ]:
prediction_test

In [ ]:
prediction_test = sc_test_target.inverse_transform(prediction_test)
y = sc_test_target.inverse_transform(y)

In [ ]:
rmse_test = []
for i in range(len(prediction_test)):
    rmse = mean_squared_error(prediction_test[i], y[i], squared=False)
    rmse_test.append(rmse)

In [ ]:
df_plot_test = pd.DataFrame({"rmse":rmse_test})
df_plot_test.plot()

In [ ]:
n = -1

In [ ]:
prediction_test01 = prediction_test[n].reshape(-1).tolist()

In [ ]:
y01 = y[n].reshape(-1).tolist()

In [ ]:
df_price_test = pd.DataFrame({"predicted price": prediction_test01,
                              "Actual price": y01})

In [ ]:
df_price_test.plot()

### save model

In [ ]:
#model.save("C:/Users/Tibame/PycharmProjects/PythonETL/Fruit Price Prediction/models/Banana_Price/Taipei1.h5")